# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


In [13]:
%%sql SELECT *
FROM Master1
WHERE CRU <> 'NULL'
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
1,Mercurey,11.5,1980,300,1,Bohn,Rene,Alsace
1,Mercurey,11.5,1980,1,73,Bourguignon,Emile,Jura
2,Julienas,11.3,1974,100,5,Joudeat,Lucien,Bourgogne
3,Savigny les Beaunes,12.1,1978,400,1,Bohn,Rene,Alsace
4,Mercurey,10.9,1980,35,10,Fournier,Jean Claude,Bourgogne
5,Pommard,11.7,1976,46,30,Desbois,Maurice,Bourgogne
6,Mercurey,11.2,1981,300,42,Juillot,Maurice,Bourgogne
7,Grands Echezeaux,11.7,1968,60,98,Berard,None,Rhone
8,Cotes de Beaune Villages,12.3,1975,12,90,Roure,Raymond,Rhone
9,Chapelle Chambertin,11.9,1973,None,None,None,None,None


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

In [14]:
%%sql SELECT *
FROM Master2;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
None,None,None,None,None,None,None,11,Breton,Andre,petit,None
None,None,None,None,None,None,None,13,Barthes,Roland,moyen,None
None,None,None,None,None,None,None,16,Balzac,Honore de,moyen,None
None,None,None,None,None,None,None,18,Celine,Louis Ferdinand,gros,None
None,None,None,None,None,None,None,20,Chateaubriand,Francois-Rene de,moyen,None
None,None,None,None,None,None,None,21,Corbiere,Tristan,petit,None
None,None,None,None,None,None,None,23,Corneille,Pierre,petit,None
None,None,None,None,None,None,None,25,Char,Rene,petit,None
None,None,None,None,None,None,None,27,Dumas,Alexandre,gros,None
None,None,None,None,None,None,None,29,Fournier,Alain,petit,None


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [15]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [16]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)


In [17]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [18]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [19]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [100]:
%%sql DROP TABLE IF EXISTS Wines;
CREATE TABLE Wines AS
SELECT DISTINCT NV, DEGRE, MILL, CRU
FROM MASTER1
WHERE NV IS NOT NULL;

DROP TABLE IF EXISTS Productors;
CREATE TABLE Productors AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

DROP TABLE IF EXISTS Production;
CREATE TABLE Production AS
SELECT NP, NV, QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [101]:
%%sql
SELECT * FROM Production

 * sqlite:///wine.db
Done.


NP,NV,QTE
1,1,300
73,1,1
5,2,100
1,3,400
10,4,35
30,5,46
42,6,300
98,7,60
90,8,12
98,10,100


#### Exercise 1.4

Create the new tables from Master2:

In [102]:
%%sql DROP TABLE IF EXISTS Clients;
CREATE TABLE Clients AS
SELECT DISTINCT NB, NOM, PRENOM, REGION,  TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

DROP TABLE IF EXISTS Orders;
CREATE TABLE Orders AS
SELECT NB, NV, DATES, QTE, LIEU
FROM MASTER2
WHERE NB IS NOT NULL AND NV IS NOT NULL

 * sqlite:///wine.db
Done.
Done.
Done.
Done.


[]

In [104]:
%sql SELECT * FROM Orders

 * sqlite:///wine.db
Done.


NB,NV,DATES,QTE,LIEU
2,1,1977-11-02,33,BORDEAUX
44,1,2015-10-16,1,PARIS
45,1,1983-12-31,1,RENNES
48,2,1983-12-25,2,LYON
7,3,1978-11-01,6,NICE
49,3,1987-12-24,5,MERCUREY
8,4,1982-11-05,12,TOULOUSE
44,4,2015-10-16,6,PARIS
50,4,1981-06-29,4,SENS
2,5,1984-11-04,50,MERCUREY


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [36]:
%%sql
SELECT DISTINCT TYPE AS Type_of_buveurs
FROM Clients

 * sqlite:///wine.db
Done.


Type_of_buveurs
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [54]:
%%sql
SELECT CRU, REGION
FROM Production AS p1 
        JOIN Productors AS p2 ON p1.NP = p2.NP
        JOIN Wines AS w ON p1.NV = w.NV
WHERE CRU IN ('Pommard', 'Brouilly')

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [57]:
%%sql
SELECT REGION
FROM Production AS p1 
        INNER JOIN Productors AS p2 ON p1.NP = p2.NP
        INNER JOIN Wines AS w ON p1.NV = w.NV 
WHERE CRU = 'Pommard' AND
REGION IN (SELECT REGION
           FROM Production AS p1 
                INNER JOIN Productors AS p2 ON p1.NP = p2.NP
                INNER JOIN Wines AS w ON p1.NV = w.NV 
          WHERE CRU = 'Brouilly')

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [61]:
%%sql
SELECT CRU, MILL, SUM(QTE) AS "Number_of_wines_bough"
FROM Orders AS o 
        INNER JOIN Wines AS w ON o.NV = w.NV
GROUP BY CRU

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bough
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1978,780
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1
Clairette de Die,1976,5
Cornas,1980,25
Cote de Brouilly,1980,23


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [78]:
%%sql
SELECT p1.NV, COUNT(p1.NP) AS "Number_of_producers"
FROM Production AS p1 
        INNER JOIN Productors AS p2 ON p1.NP = p2.NP
        INNER JOIN Wines AS w ON p1.NV = w.NV 
GROUP BY w.NV
HAVING COUNT(p1.NP) > 3

 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [95]:
%%sql
SELECT p1.NP, p1.NOM, p1.PRENOM
FROM Productors AS p1 
       LEFT JOIN Production AS p2 ON p1.NP = p2.NP
GROUP BY p1.NP
HAVING p2.NV IS NULL

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [106]:
%%sql
SELECT c.NB, c.NOM, c.PRENOM
FROM Orders AS o 
    JOIN Clients AS c ON o.NB = c.NB
    JOIN Wines AS w ON o.NV = w.NV
WHERE w.Mill = 1980
GROUP BY c.NB

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [113]:
%%sql
SELECT c.NB, c.NOM, c.PRENOM
FROM Clients AS c 
        LEFT JOIN Orders AS o ON o.NB = c.NB
WHERE c.NB NOT IN (SELECT c.NB
                    FROM Orders AS o 
                        JOIN Clients AS c ON o.NB = c.NB
                        JOIN Wines AS w ON o.NV = w.NV
                    WHERE w.Mill = 1980
                    GROUP BY c.NB)
GROUP BY c.NB



 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [116]:
%%sql
SELECT c.NB, c.NOM, c.PRENOM
FROM Orders AS o 
    JOIN Clients AS c ON o.NB = c.NB
    JOIN Wines AS w ON o.NV = w.NV
WHERE w.Mill = 1980
GROUP BY c.NB
HAVING c.NB NOT IN (SELECT c.NB
                    FROM Orders AS o 
                        JOIN Clients AS c ON o.NB = c.NB
                        JOIN Wines AS w ON o.NV = w.NV
                    WHERE w.Mill <> 1980
                    GROUP BY c.NB)
 

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [117]:
%%sql
SELECT NV, CRU, MILL, DEGRE
FROM Wines
WHERE MILL = 1980

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [120]:
%%sql
SELECT w.NV, w.CRU, w.MILL, w.DEGRE
FROM Orders AS o
    JOIN Wines AS w ON o.NV = w.NV
WHERE o.NB = 2 AND w.MILL = 1980

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [139]:
%%sql 
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM Clients AS c
WHERE NOT EXISTS (SELECT w.NV
                  FROM Wines AS w
                  WHERE w.MILL = 1980
                  AND NOT EXISTS (SELECT o.NB
                                  FROM Orders AS o
                                  WHERE w.NV=o.NV
                                  AND o.NB = c.NB
                                  )
                )

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
